In [6]:
import polars as pl
import pandas as pd
from pathlib import Path
from app.src.db.db import Database
from app.src.core.config import settings

In [7]:
db_rd = Database(settings.RAW_DATABASE_URI)

In [3]:
csv_path = Path.home().parent / 'app/src/data/input/inkar_2024.csv'

cursor = db_rd.cursor()

# Read the first line of the CSV file to extract column names
with open(csv_path, "r") as csv_file:
    header_row = csv_file.readline().strip()
    column_names = header_row.split(';')

# Construct a CREATE TABLE statement with the extracted column names
sql_create_table = f"""
    DROP TABLE IF EXISTS temporal.inkar_2024;
    CREATE TABLE temporal.inkar_2024 ({', '.join([f'{column_name} TEXT' for column_name in column_names])})
"""

db_rd.perform(sql_create_table)

# Load data from CSV into the table
with open(csv_path, "r") as csv_file:
    next(csv_file)  # Skip the header row
    cursor.copy_expert(f"COPY temporal.inkar_2024 FROM STDIN WITH CSV HEADER DELIMITER ';'", csv_file)

# Commit the transaction
db_rd.conn.commit()

# Commit and close the connection
cursor.close()

In [33]:
# aggregate inkar data -> less rows = less geoms -> smaller table

sql_create_table = """
    DROP TABLE IF EXISTS temporal.inkar_2024_aggregated;
    CREATE TABLE temporal.inkar_2024_aggregated (
        bereich TEXT, 
        id TEXT, 
        kuerzel TEXT, 
        indikator TEXT, 
        raumbezug TEXT, 
        kennziffer TEXT, 
        name TEXT, 
        jahre_werte JSONB
    );
    """
db_rd.perform(sql_create_table)

In [34]:
sql_aggregate_inkar_data = """
    INSERT INTO temporal.inkar_2024_aggregated (bereich, id, kuerzel, indikator, raumbezug, kennziffer, name, jahre_werte)
    SELECT 
        bereich, 
        id, 
        kuerzel, 
        indikator, 
        raumbezug, 
        kennziffer, 
        name,
        jsonb_agg(jsonb_build_object('zeitbezug', zeitbezug, 'wert', wert)) AS jahre_werte
    FROM 
        temporal.inkar_2024
    GROUP BY 
        bereich, 
        id, 
        kuerzel, 
        indikator, 
        raumbezug, 
        kennziffer, 
        name;
    """
db_rd.perform(sql_aggregate_inkar_data)

In [35]:
db_rd.perform("""CREATE INDEX ON temporal.inkar_2024_aggregated (kennziffer)""")

In [4]:
# db_rd.perform("""CREATE INDEX idx_regionalschlüssel_ars ON germany_municipalities (regionalschlüssel_ars);""")

In [ ]:
# db_rd.perform("""DROP INDEX idx_regionalschlüssel_ars""")

In [11]:
# Raumgliederungen des BBSR: https://www.bbsr.bund.de/BBSR/DE/forschung/raumbeobachtung/Raumabgrenzungen/raumabgrenzungen-uebersicht

# INKAR Raumbezug: 

# Arbeitsmarktregionen
# BBSR-Mittelbereiche
# Braunkohlereviere (auch nicht förderfähige)
# Bund
# Bundesländer
# EU27
# Großstadtregionaler Einzugsbereich
# Großstadtregionen
# IHK-Bezirke
# NUTS0
# NUTS1
# NUTS2
# Raumordnungsregionen
# Raumtyp Bezug: Lage (Gemeindeverbände)
# Raumtyp Bezug: Lage (Kreise)
# Regierungsbezirke
# Siedlungsstruktureller Kreistyp
# Siedlungsstruktureller Kreistyp (Bundesländer)
# Siedlungsstruktureller Kreistyp (West-Ost)
# Siedlungsstruktureller Regionstyp
# Siedlungsstruktureller Regionstyp (Bundesländer)
# Siedlungsstruktureller Regionstyp (West-Ost)
# Stadt- und Gemeindetyp
# Stadt- und Gemeindetyp (Bundesländer)
# Stadt- und Gemeindetyp (West-Ost)
# Stadt- und Gemeindetyp differenziert
# Städtischer-Ländlicher Raum
# Städtischer-Ländlicher Raum (Bundesländer)
# Städtischer-Ländlicher Raum (West-Ost)
# West-Ost
# Zentrale Orte (ja/nein)
# Zentralörtliche Einstufung (zusammengefasst)
# zentralörtliche Funktion - Grundzentrum
# zentralörtliche Funktion - Mittelzentrum
# zentralörtliche Funktion - Oberzentrum
# zentralörtliche Funktion - keine

raumbezuege = [
    'Gemeinden',
    'Gemeindeverbände (Verwaltungsgemeinschaft)',
    'Kreise',
    'Kreisregionen',
    'Metropolregionen',
    'Regionalstatistischer Raumtyp 17 (RegioStaR17)',
    'Regionalstatistischer Raumtyp 2 (RegioStaR2)',
    'Regionalstatistischer Raumtyp 7 (RegioStaR7)',
    'Regionalstatistischer Raumtyp Gemeindetyp 5 (Regio'               
    ]

In [21]:
# Values to exclude
# maybe raumbezug sinnvoller
exclude_values = {
    '0', 
    '01',
    '001', '002', '003', '004', '005', '006', '007', '008', '009', '01', '010', '0101', '0102', '0103', '0104', '0105',
    '1', '06', '05', '14', '11', '15', '09', '12', '07', '08', '02', '16', '13', '10', '04', '03', '077', '074', '102', '139', '155', '041', '040', '033', '022', '057', '085', 
    '046', '056', '087', '148', '106', '105', '013', '066', '047', '060', '029', '020', '030', '125', '038', '061', '027', '079', '137', '043', '107', '150', '037', '158', '090', 
    '044', '071', '124', '011', '016', '146', '096', '069', '065', '088', '138', '075', '091', '059', '058', '031', '014', '145', '126', '112', '159', '149', '036', '101', '113', 
    '055', '100', '064', '068', '082', '103', '110', '080', '053', '152', '050', '099', '045', '081', '051', '024', '157', '054', '012', '111', '034', '018', '035', '129', '063', 
    '093', '092', '143', '026', '015', '109', '019', '121', '151', '049', '073', '072', '039', '120', '094', '095', '154', '052', '097', '032', '028', '042', '086', '134', '156', 
    '133', '131', '130', '023', '135', '078', '160', '140', '083', '147', '070', '119', '084', '089', '153', '136', '098', '116', '122', '062', '025', '067', '021', '104', '132', 
    '123', '114', '142', '144', '108', '118', '128', '076', '141', '048', '017', '115', '117', '127', '0150', '1550', '0501', '0705', '1440', '0806', '0904', '0808', '0308', '1301', 
    '1404', '0502', '1402', '0630', '1530', '1603', '1450', '1330', '0940', '0601', '0703', '0913', '0130', '0911', '1210', '0850', '0914', '0508', '0401', '1220', '0603', '1502', 
    '1204', '1401', '0320', '0503', '0505', '1320', '0907', '1110', '0917', '0909', '0803', '0640', '0311', '0912', '0910', '1203', '0750', '0903', '0704', '1504', '0312', '0811', 
    '0804', '0650', '0602', '1503', '1205', '0350', '0340', '0908', '0410', '0303', '0506', '1250', '0730', '0210', '0830', '0510', '1340', '1602', '0509', '0802', '0310', '0507', 
    '0801', '0820', '1303', '1010', '1510', '0512', '1101', '0504', '0330', '1020', '0301', '0930', '1304', '0605', '0313', '1410', '0901', '0905', '1420', '0807', '0840', '0720', 
    '0916', '1001', '0902', '0110', '0520', '0918', '0540', '0550', '1030', '1350', '0530', '0120', '0805', '0306', '0809', '0701', '0511', '0302', '1403', '1520', '1604', '0620', 
    '0906', '1202', '0812', '1310', '0915', '0610', '0810', '0513', '0305', '0950', '1430', '1302', '1040', '0140', '0201', '0304', '1540', '0307', '1201', '0702', '1501', '1601', 
    '0740', '0920', '1230', '1240', '0710', '0309', '0604',
    '010101', '010102', '010103', '010104', '010105', '010106', '010108', '010109', '010110', '010111', '010112', '010113', '010201', '010202', '010203', '010204',
    '010301', '010302', '010303', '010304', '010305', '010401', '010402', '010403', '010404', '010501', '010502', '010503', '010504', '010505', '010506', '010507',
    '161', '1610', '162', '1620', '163', '1630', '164', '1640', '165', '1650', '166', '167', '168', '169', '17', '170', '171', '172', '173', '174', '175', '176', '177', 
    '178', '179', '18', '180', '181', '182', '183', '184', '185', '186', '187', '188', '189', '19', '190', '191', '192', '193', '194', '195', '196', '197', '198', '199', 
    '2', '20', '200', '201', '202', '203', '204', '205', '206', '207', '208', '209', '21', '210', '211', '212', '213', '214', '215', '216', '217', '218', '219', '22', 
    '220', '221', '222', '223', '224', '225', '23', '230', '24', '240', '25', '250', '26', '27', '28', '29', '3', '30', '31', '32', '33', '34', '35', '36', '37', '38', 
    '39', '4', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', 
    '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '99', '999', '99999999', 'AT', 'AT1', 'AT11', 'AT12', 'AT13', 'AT2', 'AT21', 
    'AT22', 'AT3', 'AT31', 'AT32', 'AT33', 'AT34', 'BE', 'BE1', 'BE10', 'BE2', 'BE21', 'BE22', 'BE23', 'BE24', 'BE25', 'BE3', 'BE31', 'BE32', 'BE33', 'BE34', 'BE35', 'BG', 
    'BG3', 'BG31', 'BG32', 'BG33', 'BG34', 'BG4', 'BG41', 'BG42', 'CY', 'CY0', 'CY00', 'CZ', 'CZ0', 'CZ01', 'CZ02', 'CZ03', 'CZ04', 'CZ05', 'CZ06', 'CZ07', 'CZ08', 'DE', 'DE1', 
    'DE11', 'DE12', 'DE13', 'DE14', 'DE2', 'DE21', 'DE22', 'DE23', 'DE24', 'DE25', 'DE26', 'DE27', 'DE3', 'DE30', 'DE4', 'DE40', 'DE5', 'DE50', 'DE6', 'DE60', 'DE7', 'DE71', 
    'DE72', 'DE73', 'DE8', 'DE80', 'DE9', 'DE91', 'DE92', 'DE93', 'DE94', 'DEA', 'DEA1', 'DEA2', 'DEA3', 'DEA4', 'DEA5', 'DEB', 'DEB1', 'DEB2', 'DEB3', 'DEC', 'DEC0', 'DED', 
    'DED2', 'DED4', 'DED5', 'DEE', 'DEE0', 'DEF', 'DEF0', 'DEG', 'DEG0', 'DK', 'DK0', 'DK01', 'DK02', 'DK03', 'DK04', 'DK05', 'EE', 'EE0', 'EE00', 'EL', 'EL3', 'EL30', 'EL4', 
    'EL41', 'EL42', 'EL43', 'EL5', 'EL51', 'EL52', 'EL53', 'EL54', 'EL6', 'EL61', 'EL62', 'EL63', 'EL64', 'EL65', 'ES', 'ES1', 'ES11', 'ES12', 'ES13', 'ES2', 'ES21', 'ES22', 
    'ES23', 'ES24', 'ES3', 'ES30', 'ES4', 'ES41', 'ES42', 'ES43', 'ES5', 'ES51', 'ES52', 'ES53', 'ES6', 'ES61', 'ES62', 'ES63', 'ES64', 'ES7', 'ES70', 'EU27', 'FI', 'FI1', 'FI19', 
    'FI1B', 'FI1C', 'FI1D', 'FI2', 'FI20', 'FR', 'FR1', 'FR10', 'FRB', 'FRB0', 'FRC', 'FRC1', 'FRC2', 'FRD', 'FRD1', 'FRD2', 'FRE', 'FRE1', 'FRE2', 'FRF', 'FRF1', 'FRF2', 'FRF3', 
    'FRG', 'FRG0', 'FRH', 'FRH0', 'FRI', 'FRI1', 'FRI2', 'FRI3', 'FRJ', 'FRJ1', 'FRJ2', 'FRK', 'FRK1', 'FRK2', 'FRL', 'FRL0', 'FRM', 'FRM0', 'FRY', 'FRY1', 'FRY2', 'FRY3', 'FRY4', 
    'FRY5', 'HR', 'HR0', 'HR02', 'HR03', 'HR05', 'HR06', 'HU', 'HU1', 'HU11', 'HU12', 'HU2', 'HU21', 'HU22', 'HU23', 'HU3', 'HU31', 'HU32', 'HU33', 'IE', 'IE0', 'IE04', 'IE05', 
    'IE06', 'IT', 'ITC', 'ITC1', 'ITC2', 'ITC3', 'ITC4', 'ITF', 'ITF1', 'ITF2', 'ITF3', 'ITF4', 'ITF5', 'ITF6', 'ITG', 'ITG1', 'ITG2', 'ITH', 'ITH1', 'ITH2', 'ITH3', 'ITH4', 'ITH5', 
    'ITI', 'ITI1', 'ITI2', 'ITI3', 'ITI4', 'LT', 'LT0', 'LT01', 'LT02', 'LU', 'LU0', 'LU00', 'LV', 'LV0', 'LV00', 'MT', 'MT0', 'MT00', 'NL', 'NL1', 'NL11', 'NL12', 'NL13', 'NL2', 
    'NL21', 'NL22', 'NL23', 'NL3', 'NL31', 'NL32', 'NL33', 'NL34', 'NL4', 'NL41', 'NL42', 'PL', 'PL2', 'PL21', 'PL22', 'PL4', 'PL41', 'PL42', 'PL43', 'PL5', 'PL51', 'PL52', 'PL6', 
    'PL61', 'PL62', 'PL63', 'PL7', 'PL71', 'PL72', 'PL8', 'PL81', 'PL82', 'PL84', 'PL9', 'PL91', 'PL92', 'PT', 'PT1', 'PT11', 'PT15', 'PT16', 'PT17', 'PT18', 'PT2', 'PT20', 'PT3', 
    'PT30', 'RO', 'RO1', 'RO11', 'RO12', 'RO2', 'RO21', 'RO22', 'RO3', 'RO31', 'RO32', 'RO4', 'RO41', 'RO42', 'SE', 'SE1', 'SE11', 'SE12', 'SE2', 'SE21', 'SE22', 'SE23', 'SE3', 'SE31', 
    'SE32', 'SE33', 'SI', 'SI0', 'SI03', 'SI04', 'SK', 'SK0', 'SK01', 'SK02', 'SK03', 'SK04'
    }

# Fetch kennziffer from the database
kennziffern = db_rd.select(f"""SELECT DISTINCT kennziffer FROM temporal.inkar_2024_aggregated WHERE raumbezug in ({', '.join([f"'{rb}'" for rb in raumbezuege])})""")
kennziffern = [kennziffer[0] for kennziffer in kennziffern]

# Filter out excluded values
kennziffern = [k for k in kennziffern if k not in exclude_values]

In [22]:
sql_create_inkar_table_with_geom = """
    DROP TABLE IF EXiSTS temporal.inkar_2024_with_geom;
    CREATE TABLE temporal.inkar_2024_with_geom AS
    SELECT t.*, NULL::geometry AS geom
    FROM temporal.inkar_2024_aggregated t
    LIMIT 0;
"""
db_rd.perform(sql_create_inkar_table_with_geom)    

In [ ]:
for count, kennziffer in enumerate(kennziffern):
    print(f"{count} out of {len(kennziffern)}")
    sql_insert_data = f"""
        INSERT INTO temporal.inkar_2024_with_geom (bereich, id, kuerzel, indikator, raumbezug, kennziffer, name, jahre_werte, geom)
        SELECT 
            t.bereich, 
            t.id, 
            t.kuerzel, 
            t.indikator, 
            t.raumbezug, 
            t.kennziffer, 
            t.name, 
            t.jahre_werte, 
            (SELECT ST_Multi(ST_Union(geom)) FROM germany_municipalities WHERE regionalschlüssel_ars LIKE '{kennziffer}%') AS geom
        FROM 
            temporal.inkar_2024_aggregated t
        WHERE 
            t.kennziffer = '{kennziffer}';
    """    
    db_rd.perform(sql_insert_data)

In [24]:
db_rd.perform("""CREATE INDEX ON temporal.inkar_2024_with_geom USING GIST (geom)""")

Eigentlich müsste ich einen Table pro Jahr machen, da die Gemeinden ja neu aufgeteilt werden etc.

In [ ]:
# Arbeitslose auf Gemeindeebene

SELECT 
    t.bereich, 
    t.id, 
    t.kuerzel, 
    t.indikator, 
    t.raumbezug, 
    t.kennziffer, 
    t.name, 
    t.jahre_werte, 
    t.geom,
    (jsonb_array_elements(t.jahre_werte) ->> 'zeitbezug')::int AS jahr,
    (regexp_replace(jsonb_array_elements(t.jahre_werte) ->> 'wert', ',', '', 'g'))::numeric AS wert
FROM 
    temporal.inkar_2024_with_geom t
WHERE 
    t.indikator = 'Studierende'
	and raumbezug in ('Gemeindeverbände (Verwaltungsgemeinschaft)');

In [ ]:
# Potenzielle INKAR-Indikatoren auf Gemeindeebene:

5- und mehr Raum-Wohnungen
ALG II-Leistungen an SGBII
Abhängigenquote Alte
Abhängigenquote Junge
Alleinerziehende erwerbsfähige Leistungsberechtigte
Allgemeinärzte
Anteil Minijobs (Frauen)
Anteil Minijobs (Männer)
Anteil Minijobs (Nebenverdienst)
Anteil Minijobs (Nebenverdienst) an den Beschäftigungsverhältnissen
Anteil Minijobs (ausschließlich) an den Beschäftigungsverhältnissen
Anteil Minijobs (ausschließlich) an geringfügig Beschäftigten
Anteil Minijobs an den Beschäftigungsverhältnissen
Anteil jüngere Arbeitslose
Anteil männliche jüngere Arbeitslose
Anteil männliche ältere Arbeitslose
Anteil weibliche jüngere Arbeitslose
Anteil weibliche ältere Arbeitslose
Anteil ältere Arbeitslose
Arbeitslose
Arbeitslose Frauen
Arbeitslose Männer
Arbeitslose mit Anforderungsniveau Experte
Arbeitslose mit Anforderungsniveau Fachkraft
Arbeitslose mit Anforderungsniveau Helfer
Arbeitslose mit Anforderungsniveau Spezialist
Arbeitslose ohne Ausbildung
Ausgaben für Sachinvestitionen
Ausländische Arbeitslose
Ausländische Studierende
Ausländische männliche Arbeitslose
Ausländische weibliche Arbeitslose
Auspendler
Bandbreitenverfügbarkeit mindestens 1.000 Mbit/s
Bandbreitenverfügbarkeit mindestens 100 Mbit/s
Bandbreitenverfügbarkeit mindestens 50 Mbit/s
Baugenehmigungen für Wohnungen
Baugenehmigungen für Wohnungen in Ein- und Zweifamilienhäusern
Baugenehmigungen für Wohnungen in Mehrfamilienhäusern
Bedarfsgemeinschaften mit Kindern
Beschäftigtenquote
Beschäftigtenquote Frauen
Beschäftigtenquote Männer
Bevölkerung (mit Korrektur VZ 1987/Zensus 2011)
Bevölkerung gesamt
Bevölkerung in Mittelzentren
Bevölkerung in Oberzentren
Bevölkerung männlich
Bevölkerung weiblich
Bevölkerungsentwicklung (10 Jahre)
Bevölkerungsentwicklung (5 Jahre)
Bodenfläche gesamt qkm
Durchschnittsalter der Bevölkerung
Ein- und Zweifamilienhäuser
Ein- und Zweiraumwohnungen
Einkommensteuer
Einpendler
Einpersonen-Bedarfsgemeinschaften
Einpersonenhaushalte
Einwohner 65 Jahre und älter
Einwohner 75 Jahre und älter
Einwohner unter 3 Jahren
Einwohner unter 6 Jahre
Einwohner von 18 bis unter 25 Jahren
Einwohner von 25 bis unter 30 Jahren
Einwohner von 3 bis unter 6 Jahren
Einwohner von 30 bis unter 50 Jahren
Einwohner von 50 bis unter 65 Jahren
Einwohner von 6 bis unter 18 Jahren
Einwohner von 65 bis unter 75 Jahren
Einwohner-Arbeitsplatz-Dichte
Einwohnerdichte
Einzelhandelsrelevante Kaufkraft
Erholungsfläche
Erholungsfläche je Einwohner
Erreichbarkeit von Autobahnen
Erreichbarkeit von Flughäfen
Erreichbarkeit von IC/EC/ICE-Bahnhöfen
Erreichbarkeit von Mittelzentren
Erreichbarkeit von Oberzentren
Erwerbsfähige Bevölkerung (15 bis unter 65 Jahre)
Erwerbsfähige Leistungsberechtigte
Erwerbsfähige Leistungsberechtigte (Frauen)
Fertiggestellte Wohnungen je Wohnung im Bestand
Flächenneuinanspruchnahme
Frauenanteil
Frauenanteil 20 bis unter 40 Jahre
Freifläche
Freifläche je Einwohner
Geborene
Gesamtwanderungssaldo
Gestorbene
Getötete im Straßenverkehr
Gewerbesteuer
Große Bedarfsgemeinschaften
Gästeübernachtungen in Beherbergungsbetrieben
Haushalte mit Kindern
Haushalte mit hohem Einkommen
Haushalte mit mittlerem Einkommen
Haushalte mit niedrigem Einkommen
Haushaltsgröße
Hausärzte
Internisten
Junge erwerbsfähige Leistungsberechtigte
Jüngere Arbeitslose
Kaufkraft
Kinderarmut
Kinderärzte
Landwirtschaftsfläche
Langzeitarbeitslose
Leistungen für Unterkunft an SGBII
Ländlichkeit
Mehrfamilienhäuser
Männliche Langzeitarbeitslose
Männliche Studierende
Männliche jüngere Arbeitslose
Männliche ältere Arbeitslose
Nahversorgung Apotheken Anteil der Bev. 1km Radius
Nahversorgung Apotheken Durchschnittsdistanz
Nahversorgung Grundschulen Anteil der Bev. 1km Radius
Nahversorgung Grundschulen Durchschnittsdistanz
Nahversorgung Haltestellen des ÖV Anteil der Bev. 1km Radius
Nahversorgung Haltestellen des ÖV Durchschnittsdistanz
Nahversorgung Supermärkte Anteil der Bev. 1km Radius
Nahversorgung Supermärkte Durchschnittsdistanz
Naturnähere Fläche
Naturnähere Fläche je Einwohner
Natürlicher Saldo
Neubauwohnungen in Ein- und Zweifamilienhäusern
Neubauwohnungen in Ein- und Zweifamilienhäusern je Einwohner
Neubauwohnungen in Mehrfamilienhäusern
Neubauwohnungen je Einwohner
Neue Ein- und Zweifamilienhäuser
Pendler mit Arbeitsweg 150 km und mehr
Pendler mit Arbeitsweg 300 km und mehr
Pendler mit Arbeitsweg 50 km und mehr
Pendlersaldo
Personen in Bedarfsgemeinschaften
Quote ältere Beschäftigte
Schlafgelegenheiten in Beherbergungsbetrieben
Schlüsselzuweisungen
Siedlungs- und Verkehrsfläche
Siedlungs- und Verkehrsfläche je Einwohner
Siedlungsdichte in km²
Sozialversicherungspflichtig Beschäftigte am Arbeitsort
Sozialversicherungspflichtig Beschäftigte am Wohnort
Steuereinnahmen
Steuerkraft
Straßenverkehrsunfälle
Studierende
Studierende an FH
Studierende im 1. Semester
Studierende je 100 Einwohner 18 bis 25 Jahre
Umsatzsteuer
Verhältnis der Beschäftigungsquote von Frauen zu Männern
Verhältnis junge zu alte Erwerbsfähige
Verunglückte im Straßenverkehr
Verweildauer in Beherbergungsbetrieben
Waldfläche
Wasserfläche
Weibliche Einwohner 65 Jahre und älter
Weibliche Einwohner 75 Jahre und älter
Weibliche Einwohner von 18 bis unter 25 Jahren
Weibliche Einwohner von 25 bis unter 30 Jahren
Weibliche Einwohner von 65 bis unter 75 Jahren
Weibliche Langzeitarbeitslose
Weibliche Studierende
Weibliche jüngere Arbeitslose
Weibliche ältere Arbeitslose
Wohnungen in Ein- und Zweifamilienhäusern
Wohnungen in Mehrfamilienhäusern
Wohnungsnahe Grundversorgung Apotheke
Wohnungsnahe Grundversorgung Grundschule
Wohnungsnahe Grundversorgung Hausarzt
Wohnungsnahe Grundversorgung Supermarkt
Zuweisungen für Investitionsfördermaßnahmen
Ältere Arbeitslose
Ältere erwerbsfähige Leistungsberechtigte
Ärzte